In [2]:
import os
import wget
import os
import glob
import pickle
import pandas as pd
import numpy as np
from dask.diagnostics import ProgressBar
from arboreto.utils import load_tf_names
from arboreto.algo import grnboost2
from ctxcore.rnkdb import FeatherRankingDatabase as RankingDatabase
from pyscenic.utils import modules_from_adjacencies, load_motifs
from pyscenic.prune import prune2df, df2regulons
from pyscenic.aucell import aucell
import seaborn as sns
from pyscenic.export import export2loom
import sys
import loompy as lp
import numpy as np

os.chdir('./scenic')

In [39]:
MOTIF_ANNOTATIONS_FNAME = 'motifs-v9-nr.mgi-m0.001-o0.0.tbl'
MM_TFS_FNAME = 'mm_mgi_tfs.txt'
SC_EXP_FNAME = 'E18_Mesen_counts.csv' #which had been gotten by R write.csv(t(as.matrix(Mesen@assays$RNA@counts)),file = "E18_Mesen_counts.csv")


In [40]:
ex_matrix  = pd.read_csv(SC_EXP_FNAME, index_col= 0)
tf_names = load_tf_names(MM_TFS_FNAME)

In [41]:
REGULONS_FNAME =  "regulons.p"
MOTIFS_FNAME =  "motifs_E18_Mesen.csv"

In [27]:
db_fnames  = ['mm9-tss-centered-10kb-10species.mc9nr.feather']

In [28]:
def name(fname):
     return os.path.splitext(os.path.basename(fname))[0] 

In [29]:
dbs = [RankingDatabase(fname=fname, name=name(fname)) for fname in db_fnames]

In [42]:
adjacencies = grnboost2(expression_data=ex_matrix, tf_names=tf_names, verbose=True)


preparing dask client
parsing input
creating dask graph
4 partitions
computing dask graph
shutting down client and local cluster
finished


In [43]:
modules = list(modules_from_adjacencies(adjacencies, ex_matrix))


2022-08-03 10:41:36,079 - pyscenic.utils - INFO - Calculating Pearson correlations.

2022-08-03 10:41:37,834 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modules_from_adjacencies function, or '--mask_dropouts' from the CLI.
	Dropout masking is currently set to [False].

2022-08-03 10:42:56,940 - pyscenic.utils - INFO - Creating modules.


In [44]:
with ProgressBar():
    df = prune2df(dbs, modules, MOTIF_ANNOTATIONS_FNAME)

[########################################] | 100% Completed | 12min 16.1s


In [45]:
regulons = df2regulons(df)

Create regulons from a dataframe of enriched features.
Additional columns saved: []


In [46]:
df.to_csv(MOTIFS_FNAME)

In [47]:
with open(REGULONS_FNAME, "wb") as f:
    pickle.dump(regulons, f)

In [48]:
auc_mtx = aucell(ex_matrix, regulons, num_workers=4)

In [ ]:
auc_mtx

In [19]:
from pyscenic.export import export2loom


In [50]:
export2loom(ex_mtx = ex_matrix, auc_mtx = auc_mtx, regulons = [r.rename(r.name.replace('(+)',' ('+str(len(r))+'g)')) for r in regulons], out_fname = "E18_Mesen.loom")

In [49]:
auc_mtx.to_csv('auc_E18_Mesen.csv')